## Rep Table

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm # HP filter and OLS regression
import datetime
import subprocess
from datetime import datetime as dt
import matplotlib.dates as mdates
import importlib
from pandas_datareader.fred import FredReader


In [2]:
import data_load_table

Load Data

In [3]:
series = data_load_table.get_fred_data('Q','own_def')

View Series

In [4]:
series = series.loc['1985-3-31':'2017-12-31']
series.tail()
series

,Y,C,G,I,N,BK,CO,D,W,Pi,i,tax
date,,,,,,,,,,,,
1985-03-31,8400.820,5321.159,2120.284,1169.416,96572.333333,NaN,0.82,1710731.0,314.0,49.543333,8.476667,353.018
1985-06-30,8474.787,5370.011,2167.338,1189.334,97269.666667,NaN,0.97,1774640.0,323.0,49.945000,7.923333,307.582
1985-09-30,8604.220,5472.014,2216.743,1181.226,97844.333333,NaN,1.15,1823103.0,320.0,50.337000,7.900000,340.030
1985-12-31,8668.188,5483.866,2225.878,1224.763,98428.333333,NaN,1.28,1945942.0,320.0,50.687667,8.103333,345.182
1986-03-31,8749.127,5531.807,2243.464,1224.404,98837.666667,NaN,1.35,1986816.0,321.0,51.046667,7.826667,341.825
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31,19304.352,13060.061,3389.649,3391.749,145222.333333,178353.0,2.11,19976827.0,349.0,98.946667,0.450000,1574.500
2017-03-31,19398.343,13160.475,3380.244,3382.703,145820.000000,195516.0,2.19,19846420.0,352.0,99.524000,0.700000,1579.976
2017-06-30,19506.949,13225.743,3389.811,3435.929,146383.333333,203976.0,2.11,19844554.0,354.0,99.724333,0.950000,1600.122


Apply HP filter

In [5]:
# Initialize dictionaries to store the cycle and trend components
cycle_components = {}
trend_components = {}

for name, data in series.items():
    # Apply the HP filter to each series
    cycle, trend = sm.tsa.filters.hpfilter(np.log(data).dropna(), 1600)
    
    # Store the cycle and trend components in their respective dictionaries
    cycle_components[name + '_cycle'] = cycle
    trend_components[name + '_trend'] = trend



Create table

In [6]:
# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Var', 'Std Dev', 'Cor(y, x)', 'Cor(x, x_t-1)'])

# Calculate the standard deviation and correlations for each cycle component
for name, cycle_data in cycle_components.items():
    std_dev = cycle_data.std()
    cor_y_x = cycle_data.corr(cycle_components['Y_cycle'])
    cor_x_xt_minus_1 = cycle_data.corr(cycle_data.shift(1))
    
    # Append the results to the DataFrame using pd.concat
    new_row = pd.DataFrame({
        'Var': [name.replace('_cycle', '')],  # Remove '_cycle' for the variable name
        'Std Dev': [std_dev],
        'Cor(y, x)': [cor_y_x],
        'Cor(x, x_t-1)': [cor_x_xt_minus_1]
    })
    
    results_df = pd.concat([results_df, new_row], ignore_index=True)

# Display the DataFrame
results_df = results_df.round(3)

file_path = '/Users/alexwellman/Documents/SIEPR/GitHub/macrodata-wellman/check_t/results_table.tex'
with open(file_path, 'w') as latex_file:
    latex_file.write(results_df.to_latex(index=False, float_format="%.3f"))

results_df


,Var,Std Dev,"Cor(y, x)","Cor(x, x_t-1)"
0,Y,0.010,1.000,0.880
1,C,0.009,0.883,0.876
2,G,0.012,-0.420,0.848
3,I,0.054,0.894,0.872
4,N,0.011,0.808,0.960
5,BK,0.208,-0.459,0.566
6,CO,0.173,-0.619,0.871
7,D,0.020,-0.413,0.858
8,W,0.011,-0.180,0.675
9,Pi,0.006,0.277,0.825
